# 快速开始
本指南展示了如何使用 vLLM 来：

- 对数据集运行离线批量推理；
- 为大型语言模型构建API服务器；
- 启动兼容 OpenAI 的 API 服务器。

> [默认情况下，vLLM 从HuggingFace](https://huggingface.co/)下载模型。如果您想在以下示例中使用[ModelScope](https://www.modelscope.cn/)中的模型，请设置环境变量：
```shell
export VLLM_USE_MODELSCOPE=True
```

## 离线批量推理
我们首先展示一个使用 vLLM 对数据集进行离线批量推理的示例。换句话说，我们使用 vLLM 生成输入提示列表的文本。

`LLM`从`SamplingParamsvLLM`导入。该类LLM是使用 vLLM 引擎运行离线推理的主类。该类`SamplingParams`指定采样过程的参数。


In [2]:
from vllm import LLM, SamplingParams

ModuleNotFoundError: No module named 'vllm'

定义输入提示列表和生成的采样参数。采样温度设置为0.8，核采样概率设置为0.95。有关采样参数的更多信息，请参阅类定义。

In [ ]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

初始化 `vLLM` 的引擎，以使用`LLM`类和[OPT-125M](https://arxiv.org/abs/2205.01068) 模型进行离线推理。支持的型号列表可在支持的型号中找到。

In [ ]:
llm = LLM(model="facebook/opt-125m")

调用`llm.generate`以生成输出。它将输入提示添加到 vLLM 引擎的等待队列中，并执行 vLLM 引擎以生成高吞吐量的输出。输出作为对象列表返回`RequestOutput`，其中包括所有输出标记。

In [ ]:
outputs = llm.generate(prompts, sampling_params)

# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

## OpenAI 兼容服务器
vLLM可以部署为实现`OpenAI API`协议的服务器。这使得 vLLM 可以用作使用 OpenAI API 的应用程序的直接替代品。默认情况下，它在 处启动服务器`http://localhost:8000`。您可以使用--host和参数指定地址--port。服务器当前一次托管一个模型（下面命令中的 OPT-125M）并实现列表模型、创建聊天完成和创建完成端点。我们正在积极增加对更多端点的支持。

```
python -m vllm.entrypoints.openai.api_server \
    --model facebook/opt-125m
```

默认情况下，服务器使用存储在标记生成器中的预定义聊天模板。您可以使用`--chat-template`参数覆盖此模板：

```shell
python -m vllm.entrypoints.openai.api_server \
    --model facebook/opt-125m \
    --chat-template ./examples/template_chatml.jinja
```

该服务器可以按照与OpenAI API相同的格式进行查询。例如，列出型号：

```shell
curl http://localhost:8000/v1/models
```

您可以传入参数`--api-key`或环境变量`VLLM_API_KEY`，以使服务器能够检查标头中的 API 密钥。


## 将 OpenAI Completions API 与 vLLM 结合使用
通过输入提示查询模型：
```shell
curl http://localhost:9909/v1/completions \
    -H "Content-Type: application/json" \
    -d '{
        "model": "/slurm/home/yrd/shaolab/daiyizheng/resources/modelscope/shakechen/Llama-2-7b-chat-hf",
        "prompt": "San Francisco is a",
        "max_tokens": 7,
        "temperature": 0
    }'
```

由于该服务器与 `OpenAI API` 兼容，因此您可以将其用作任何使用 `OpenAI API` 的应用程序的直接替代品。例如，查询服务器的另一种方法是通过openaipython 包：

```python
from openai import OpenAI

# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
completion = client.completions.create(model="facebook/opt-125m",
                                      prompt="San Francisco is a")
print("Completion result:", completion)
```

## Using OpenAI Chat API with vLLM
vLLM服务器旨在支持OpenAI聊天API，允许您与模型进行动态对话。聊天界面是与模型通信的一种更具交互性的方式，允许存储在聊天历史记录中的来回交换。这对于需要上下文或更详细解释的任务很有用。

使用OpenAI Chat API查询模型

您可以使用[create chat completion](https://platform.openai.com/docs/api-reference/chat/completions/create)端点在类似聊天的界面中与模型通信

```shell
curl http://localhost:8000/v1/chat/completions \
    -H "Content-Type: application/json" \
    -d '{
        "model": "facebook/opt-125m",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Who won the world series in 2020?"}
        ]
    }'
```

Python客户端示例:使用openai Python包，您还可以以类似聊天的方式与模型通信

In [ ]:
from openai import OpenAI
# Set OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

chat_response = client.chat.completions.create(
    model="facebook/opt-125m",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Tell me a joke."},
    ]
)
print("Chat response:", chat_response)